# What needs doing:

* Count all words and sentences from all terms
* Get term limits from wikipedia ✓
* Count occurences of body, desc, kinesic... items

# Questions for Tomaž/Nikola:

* How is validation run?
* Do we need to follow the per-term directory and file structure (e.g. ParlaSpeech-HR) or can we preserve the current structure?
* What is the deal with relations, coalition field `mutual` for party listing while opposition has `active` and a field `passive #government.hr`?
* I don't have the education title (e.g. `<education n="6">stručni prvostupnik sestrinstva</education>`), only years of education.
* I don't have the exact dates of terms. I can only infer them from the rest of the data I have (I estimate it from the dates of speeches.)
* I get different data for coalition-opposition formations than in the existing main TEI files for term 9.
* I need the affiliation>roles, but they are not in the MP datasheet.

# Resolved:
* ~~Is party `NZ` the same as `nezavisni`?~~ Yes, yes it is.
* ~~How to encode when an MP changes parties?~~ Like so:
```xml
<person xml:id="PernarIvan" n="1302">
                  <persName>
                     <surname>Pernar</surname>
                     <forename>Ivan</forename>
                  </persName>
                  <sex value="M"/>
                  <birth when="1985-10-14"/>
                  <education n="6">stručni prvostupnik sestrinstva</education>
                  <affiliation role="member" ref="#party.Nezavisni">
                     <roleName xml:lang="en">Member</roleName>
                  </affiliation>
                  <affiliation role="member" ref="#HS" ana="#HS.9">
                     <roleName xml:lang="en">MP</roleName>
                  </affiliation>
```
                  
Meeting notes
* Try running the perl scripts for validation
* Fill the extent fields in the main TEI.
* Term dates: to be corrected from wikipedia data, construct a new tsv and populate with data.✓
* Coalition/Opposition: Go only for `coalition` attribute in parties metadata (.xls file). Check for issues.✓
* opposition: passive seems to be always government. Is it tagged in the speeches in any parliament?✓
* Re:Education: ask Tomaž.
* Affiliation: every MP needs 2: as member of parliament and as member of their party. People like PM need only their party reference. But we do not have them in MP excell file, so this will probably cause validation issues.

Tomaž:
* Education titles and `n` parameter?
* How to date ongoing terms?

In [39]:
from typing import Union, List
from pathlib import Path
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import pandas as pd

directory = ".."
mp_path = Path(directory) / Path("Croatia_MPs_final_ 20220917.xlsx")
parties_path = Path(directory) / Path("Croatia_parties_final_20220917.xlsx")

mpdf = pd.read_excel(str(mp_path))
termdata = pd.read_csv("../terms.csv").set_index("Term")

#Correcting party:NZ to nezavisni
# mpdf["party"] = mpdf.party.replace("NZ", "nezavisni")
partiesdf = pd.read_excel(str(parties_path))
# partiesdf["party"] = partiesdf.party.str.replace(" ", "_")



In [40]:
partiesdf.loc[partiesdf.full_name.str.lower().str.contains("blok"), ["party", "full_name"]]

,party,full_name
49,BUZ,Blok umirovljenici zajedno


In [41]:
partiesdf.loc[partiesdf.full_name.str.lower().str.contains("domovinski"), ["party", "full_name"]]

,party,full_name
101,DP,Domovinski pokret


In [42]:
partiesdf.loc[partiesdf.full_name.str.lower().str.contains("rada"), ["party", "full_name"]]

,party,full_name
20,Hrvatski laburisti,Hrvatski laburisti – Stranka rada
36,Hrvatski laburisti,Hrvatski laburisti – Stranka rada
48,BM365,Bandić Milan 365 - Stranka rada i solidarnosti
56,Hrvatski laburisti,Hrvatski laburisti – Stranka rada
69,BM365,Bandić Milan 365 - Stranka rada i solidarnosti


In [43]:
mpdf.head(2)

,codemp,order_id,term1,term2,term_id,type_of_list,fullname,firstname,lastname,party,date_of_birth,year_of_birth,gender,place_of_birth,field_of_study,education_y,constituency,bp_lat,bp_lon
0,M172,1,2000-2003,4,1,normal,"Adlešič, Đurđa",Đurđa,Adlešič,HSLS,19600418,1960,1,Bjelovar,1,16,2,45.898797,16.842309
1,M418,2,2000-2003,4,2,normal,"Arlović, Mato",Mato,Arlović,SDP,19521104,1952,0,"Oštra Luka, Bosnia and Herzegovina",5,16,5,44.856287,16.662595


In [46]:
mpdf.party[~mpdf.party.isin(partiesdf.party)].unique().shape

(21,)

In [52]:
mpdf.groupby(["term2", "fullname", "party"]).count().codemp.value_counts()

1    1382
Name: codemp, dtype: int64

In [47]:
# set(mpdf.party.unique()) - set(partiesdf.party.unique())
mpdf.party[~mpdf.party.isin(partiesdf.party)].unique()

array(['HIP', 'HB', 'HND', 'SNS', 'HKDU', 'nezavisni', 'SDA HR',
       'HSP dr. Ante Starčević', 'Most', 'NZ', 'Damir Bajs NL', 'Možemo!',
       'Domovinski pokret', 'NL', nan, 'Blok', 'Hrvatski suverenisti',
       'SsIP', 'RF', 'Centar', 'HKS'], dtype=object)

# Do we have any MPs that occur in more than one parties?

In [19]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
gb = mpdf.groupby("codemp ".split()).agg({
    "party": lambda l: set(l)})
gb_problematic = gb[gb.party.apply(len) != 1]
gb_problematic.shape[0]

54

In [20]:
# problematic_codemps = gb_problematic.index.tolist()

# subset = mpdf[mpdf.codemp.isin(problematic_codemps)]
# gb = subset.groupby("fullname term1 party".split()).agg({
#     "party": lambda i: i,
    
# }
# )

# gb

# Preparing persons:

In [21]:
from xml.etree.ElementTree import Element, SubElement, tostring, XML, parse
from xml.dom import minidom
def pretty_print(s) -> None:
    print(minidom.parseString(tostring(s).decode("utf")).toprettyxml("\t"))
def pretty_string(s) -> None:
    return minidom.parseString(tostring(s).decode("utf")).toprettyxml("\t")

In [22]:
partiesdf.party.unique()

array(['DC', 'HDSSB', 'HDZ', 'HNS', 'HSLS', 'HSP', 'HSS', 'HSU', 'IDS',
       'LIBRA', 'LS', 'MDS', 'PGS', 'SBHS', 'SDAH', 'SDP', 'SDSS',
       'Hrvatski_laburisti', 'HSD', 'BDSH', 'HGS', 'HSP_AS', 'ID',
       'Naprijed_Hrvatska!', 'Novi_val', 'ORaH', 'Reformisti', 'BM365',
       'BUZ', 'HDS', 'HRAST', 'HRID', 'MOST', 'NLSP', 'PH', 'Živi_zid',
       'GLAS', 'SMSH', 'SNAGA', 'DP', 'ZK', 'SIP', 'Pametno', 'Fokus',
       'DZMH', 'KS', 'UARH'], dtype=object)

In [23]:
termdata

,From,To
Term,,
5,2003-12-22,2007-10-12
6,2008-01-11,2011-10-28
7,2011-12-22,2015-09-25
8,2015-12-03,2016-07-15
9,2016-10-14,2022-05-18
10,2022-07-22,ongoing


# Terms (line 438):

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/org[2]/listEvent`

In [24]:
listEvent = Element("listEvent")
head = SubElement(listEvent, "head")
head.set("xml:lang", "hr")
head.text = "Mandatno obdoblje"
head = SubElement(listEvent, "head")
head.set("xml:lang", "en")
head.text = "Legislative period"

for i, row in termdata.iterrows():
    event = SubElement(listEvent, "event")
    event.set("xml:id", f"HS.{i}")
    event.set("from", row["From"])
    event.set("to", row["To"])
    
    label = SubElement(event, "label")
    label.set("xml:lang", "hr")
    label.text = f"{i}. mandat"
    label = SubElement(event, "label")
    label.set("xml:lang", "en")
    label.text = f"Term {i}"
listEvent_str = pretty_string(listEvent)
listEvent_str = "\n".join(listEvent_str.split("\n")[1:])
# print(listEvent_str)

# Parties: line 447

path:
`/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/org[3:]`

In [25]:
partiesdf["id"] = partiesdf.party.apply(lambda s: f"party.{s.replace(' ', '_').upper()}")

partydata = partiesdf["party full_name id".split()].drop_duplicates()

orgs = []
for i, row in partydata.iterrows():
    org = Element("org")
    org.set("xml:id", row["id"])
    org.set("role", "parliamentaryGroup")
    orgName = SubElement(org, "orgName")
    orgName.set("full", "yes")
    orgName.set("xml:lang", "hr")
    orgName.text = row["full_name"]
    
    orgName = SubElement(org, "orgName")
    orgName.set("full", "abb")
    orgName.text = row["party"]
    orgs.append(org)
orgs = [pretty_string(i) for i in orgs]

orgstring = "\n".join(
    ["\n".join(org.split("\n")[1:])for org in orgs]
)
# print(orgstring)

# People: line 525

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listPerson`

In [26]:
people = mpdf.fullname.drop_duplicates()
assert people.shape[0] == mpdf.codemp.unique().shape[0], "People names are not unique!"
listPerson = []
for i, p in enumerate(people):
    subset = mpdf.query(f"fullname=='{p}'").reset_index(drop=True)
    surname = subset.lastname[0]
    forename = subset.firstname[0]
    sex = "M" if subset.gender[0] == 0 else "F"
    birth = str(subset.date_of_birth[0])
    birth = f"{birth[0:4]}-{birth[4:6]}-{birth[-2:]}"
    reference_name = f"{surname.replace(' ', '')}{forename.replace(' ', '')}"
    
    person = Element("person")
    person.set("xml:id", reference_name)
    persName = SubElement(person, "persName")
    
    buf = SubElement(persName, "surname")
    buf.text = surname
    
    buf = SubElement(persName, "forename")
    buf.text = forename
    
    buf = SubElement(person, "sex")
    buf.set("value", sex)
    
    buf = SubElement(person, "birth")
    buf.set("when", birth)
    
    
    for i, row in subset.iterrows():        
        ana = SubElement(person, "affiliation")
        ana.set("role", "member")
        # if f"#party.{row['party']}" == "#party.Hrvatski suverenisti":
        #     print(subset)
        ana.set("ref", f"#party.{row['party']}".replace(" ", ""))
        
        rolename = SubElement(ana, "roleName")
        rolename.set("xml:lang", "en")
        rolename.text = "Member"

        ana = SubElement(person, "affiliation")
        ana.set("role", "member")
        ana.set("ref", "#HS")
        ana.set("ana", f"#HS.{row['term2']}")
        rolename = SubElement(ana, "roleName")
        rolename.set("xml:lang", "en")
        rolename.text = "MP"
    listPerson.append(person)


In [27]:
# partiesdf.apply(lambda row: row["party"] + "->" + row["full_name"], axis=1).unique()

In [28]:
# set(mpdf.party.unique()) - set(partiesdf.party.unique())

In [29]:
listPerson = ["\n".join(pretty_string(p).split("\n")[1:]) for p in listPerson]
listPerson = "\n".join(listPerson)


In [30]:
path_to_term_tmxs = Path("../T/")
assert path_to_term_tmxs.exists()


xiincludes = ""
for file in sorted(list(path_to_term_tmxs.glob("ParlaMint-HR_T*.xml"))):
    xiincludes += f"""    <xi:include xmlns:xi="http://www.w3.org/2001/XInclude" href="{file.name}"/>\n"""


# Relations

path: `/teiCorpus/teiHeader/profileDesc/particDesc/listOrg/listRelation`

In [31]:
gb = partiesdf.groupby(["term2", "coalition"]).agg(
    {
        "id": lambda l: " ".join(set(["#"+i for i in l]))
    }
)
terms = partiesdf.term2.unique()
listRelation = ""
for term in terms:
    fromto = termdata.loc[term, :]
    ind = (term, 1)
    coalition = gb.loc[ind].id
    ind = (term, 0)
    opposition = gb.loc[ind].id
    listRelation += f"""<relation name="coalition"
            mutual="{coalition}"
            from="{fromto.From}"
            to="{fromto.To}"
            ana="#HS.{term}"/>
    <relation name="oposition"
            active="{opposition}"
            passive="#government.HR"
            from="{fromto.From}"
            to="{fromto.To}"
            ana="#HS.{term}"/>\n"""    

# Counting extents and tagUsages

In [32]:
from typing import Union
from pathlib import Path
def get_extent(path: Union[str, Path]):
    if isinstance(path, Path):
        assert path.exists()
        path = str(path)
    namespace= {
        "n": "http://www.tei-c.org/ns/1.0",
        
    }
    termtei = parse(path)
    measures = termtei.getroot().findall(".//n:measure", namespaces=namespace)
    for i in measures:
        if i.get("unit") == "speeches":
            speeches = i.get("quantity")
        if i.get("unit") == "words":
            words = i.get("quantity")
    return {"words": int(words), "speeches": int(speeches)}

def get_tagusage(path: Union[str, Path]):
    if isinstance(path, Path):
        assert path.exists()
        path = str(path)
    namespace= {
        "n": "http://www.tei-c.org/ns/1.0",
        
    }
    termtei = parse(path)
    tagusages = termtei.getroot().findall(".//n:tagUsage", namespaces=namespace)
    for i in tagusages:
        if i.get("unit") == "speeches":
            speeches = i.get("quantity")
        if i.get("unit") == "words":
            words = i.get("quantity")
    return {i.get("gi"): i.get("occurs") for i in tagusages}

path_to_term_tmxs = Path("../T/")
assert path_to_term_tmxs.exists()


extents = []
tagusages = []
for file in sorted(list(path_to_term_tmxs.glob("ParlaMint-HR_T*.xml"))):
    extents.append(get_extent(file))
    tagusages.append(get_tagusage(file))

tagusage = ""
import pandas as pd
tus = pd.DataFrame(data=tagusages).astype(int)
for gi, occurs in tus.sum().items():
    tagusage += f"""<tagUsage gi="{gi}" occurs="{occurs}"/>\n"""
extent_count = pd.DataFrame(data=extents).astype(int).sum()
words = extent_count["words"]
speeches = extent_count["speeches"]
extent = f"""<measure unit="speeches" quantity="{speeches}" xml:lang="hr">{f'{speeches:,d}'.replace(',','.')} govora</measure>
<measure unit="speeches" quantity="{speeches}" xml:lang="en">{speeches:,d} speeches</measure>
<measure unit="words" quantity="{words}" xml:lang="hr">{f'{words:,d}'.replace(',','.')} riječi</measure>
<measure unit="words" quantity="{words}" xml:lang="en">{words:,d} words</measure>
"""

# Putting it all together

In [ ]:
with open("../T/ParlaMint-HR_template2.xml") as f:
    content = f.read()
from string import Template
template = Template(content)
filled_in = template.substitute(
                    listEvent=listEvent_str,
                    orgs=orgstring,
                    listPerson = listPerson,
                    xiincludes = xiincludes,
                    listRelation = listRelation,
                    extent = extent,
                    tagusage = tagusage
                    )

with open("../T/ParlaMint-HR.xml", "w") as f:
    f.write(
        filled_in
    )